# Работа со строковыми значениями

## Лабораторная работа 6

### Форматирование строк

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

1. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [2]:
# загрузка данных из файла
recipes = pd.read_csv('data/recipes_sample.csv')

In [3]:
# выборка 5 случайных рецептов
recipes_sample = recipes.sample(5)

In [4]:
# форматирование строк и вывод таблицы
print("|{:^14}|{:^11}|".format("id", "minutes"))
print("|" + "-"*26 + "|")
for i in range(len(recipes_sample)):
    print("|    {:<10}|    {:<7}|".format(recipes_sample.iloc[i]["id"], recipes_sample.iloc[i]["minutes"]))

|      id      |  minutes  |
|--------------------------|
|    131087    |    30     |
|    201496    |    30     |
|    103892    |    135    |
|    451177    |    70     |
|    124320    |    45     |


In [5]:
# Дополнительный вариант решения с памощью Pandas
print(recipes[['id', 'minutes']].sample(5).to_markdown(index=False, 
                                                       tablefmt="orgtbl", 
                                                       numalign="center"))

|   id   |  minutes  |
|--------+-----------|
| 142732 |    10     |
| 266928 |    320    |
| 19162  |    35     |
| 202058 |    35     |
| 133077 |    18     |


In [6]:
# Дополнительный вариант решения с помощью модуля tabulate
from tabulate import tabulate

print(tabulate(recipes[['id', 'minutes']].sample(5), 
               headers='keys', 
               tablefmt="github", 
               showindex=False, 
               numalign="center"))

|   id   |  minutes  |
|--------|-----------|
| 495181 |    40     |
| 246632 |    245    |
| 458009 |    20     |
| 325903 |    40     |
| 117453 |    20     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [7]:
# функция форматированного вывода информации о рецепте
def show_info(name, steps, minutes, author_id):
    name = f'"{name.title()}"'
    steps_str = ''
    for step in steps:
        steps_str += f'\n{steps.index(step)+1}. ' + step.capitalize()

    # формирование итоговой строки
    result = f"{name}\n{steps_str}\n{'-'*10}\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут\n"

    return result

In [8]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

In [9]:
# Открываем файл с данными и считываем его содержимое
with open('data\steps_sample.xml', 'r') as xml_data:
    steps_sample = xml_data.read()

# Используем библиотеку BeautifulSoup для парсинга XML
Bs_data = BeautifulSoup(steps_sample, "xml")

# Создаем словарь, где ключами являются id рецептов, а значениями - списки шагов приготовления
recipes_steps = {}
for recipe in Bs_data.find_all('recipe'):
    recipes_steps[int(recipe.find('id').text)] = [step.text for step in recipe.find_all('step')]

In [10]:
# Создаем DataFrame из словаря recipes_steps
steps = pd.DataFrame({"recipe_id": recipes_steps.keys(), "steps": recipes_steps.values()})

In [11]:
# Объединяем два DataFrame по столбцу "id"/"recipe_id"
recipe_steps_df = pd.DataFrame.merge(recipes, 
                                  steps, 
                                  left_on='id', 
                                  right_on='recipe_id')[['name', 'id', 'steps', 'minutes', 'contributor_id']]

# Переименовываем столбец "contributor_id" в "author_id"
recipe_steps_df = recipe_steps_df.rename(columns={'contributor_id':'author_id'})

# Устанавливаем столбец "id" в качестве индекса
recipe_steps_df = recipe_steps_df.set_index('id')

In [12]:
# конвертируем данные строки с id = 170895 в словарь
recipe_info = recipe_steps_df.loc[170895].to_dict()

# вызываем функцию show_info
print(show_info(**recipe_info))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [13]:
# шаблон регулярного выражения
pattern = r'(\d+\shours?|\d+\sminutes?)'

# Ищем все совпадения шаблона в каждой строке шага
for string in recipe_steps_df.loc[25082]['steps']:
    matches = re.findall(pattern, string)
    if matches:
        print(', '.join(matches))

20 minutes
10 minutes
2 hours
10 minutes
20 minutes, 30 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [14]:
# заменяем пустые значения на пустые строки
recipes['description'] = recipes['description'].fillna('')

# шаблон регулярного выражения
pattern = r'^this[\w\s]*,?\s?but\b'

# находим рецепты, в которых найден шаблон
matching_recipes = recipes[recipes['description'].str.contains(pattern)]

# выводим количество рецептов и примеры описаний
print("Количество рецептов, в которых найден шаблон: ", len(matching_recipes))

print('\n', "Примеры:", '\n')
for description in matching_recipes['description'].sample(3):
    print(re.sub(pattern, '\033[1;31m\\g<0>\033[0m', description), end='\n\n')

Количество рецептов, в которых найден шаблон:  235

 Примеры: 

this recipe is not exactly like they have around here, but i am working on it.  this will give you a really good idea of what we bring home from a donut shop in and around houston.  these breakfast treats come from the czech immigrants who settled the texas hill country (between houston and austin and around austin), and frequently come in a fruit type (dimpled and filled, like a danish) that is also available up north, and meat (which apparently doesn't exist anywhere else but around here in texas).  ham and cheese has always been one of my favorites.  the amount of flour you use will greatly depend on whether you are making this in or around houston right before or after a rainstorm, as i always seem to; or if you are making it someplace with a normal, low level of humidity.  start the sponge the night before.

this is a very different dessert but for gorgonzola lovers it's a special treat.

this classic chinese dish is 

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [15]:
# получаем текст шагов рецепта с нужным id
steps = recipe_steps_df.loc[72367]['steps']

# заменяем для каждого шага пробелы перед и после символов дробей
for i in range(len(steps)):
    steps[i] = re.sub(r'\s*/\s*', '/', steps[i])

# выводим текст шагов после изменения
print('\n'.join(steps))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [16]:
# Создаем пустое множество для хранения уникальных слов
unique_words = set()

# Итерируемся по столбцу "steps" в DataFrame
for steps in recipe_steps_df['steps']:
    # Объединяем список строк "steps" в одну большую строку, разделяя пробелами
    steps_str = ' '.join(steps)
    # Токенизируем строку "steps_str", приводим все токены к нижнему регистру и фильтруем только слова
    words = [token.lower() for token in nltk.word_tokenize(steps_str) if token.isalpha()]
    # Добавляем все уникальные слова из списка "words" в множество "unique_words"
    unique_words.update(words)        

# вывод количества уникальных слов на экран
print(f'Кол-во уникальных слов среди всех рецептов: {len(unique_words)}')

Кол-во уникальных слов среди всех рецептов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [17]:
# Определяем функцию, которая принимает текстовую строку и возвращает количество предложений в этой строке
def len_sentences(text):
    return len(nltk.tokenize.sent_tokenize(text))

# Применяем функцию len_sentences() к столбцу 'description' 
recipes['len_sentences'] = recipes['description'].apply(len_sentences)

# Находим 5 рецептов с наибольшим количеством предложений в столбце 'description'
recipes.nlargest(5, 'len_sentences')

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,len_sentences
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,76
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,27
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,24
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,23
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,23


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [18]:
def pos_info(sentence):
    # Получаем список кортежей (слово, часть речи) для каждого слова в предложении
    pos_tags = nltk.pos_tag(nltk.word_tokenize(sentence))
    
    # Создаем пустые списки для строк с тегами и словами
    tags_line = []
    words_line = []
    
    # Для каждого кортежа (слово, часть речи) добавляем в соответствующие списки
    # отформатированные строки с тегом и словом
    for token, pos in pos_tags:
        # Определяем максимальную длину между словом и тегом
        max_len = max(len(token), len(pos))
        
        # Добавляем в списки отформатированную строки, выровненные по центру
        tags_line.append('{:{align}{width}}'.format(pos, align='^', width=max_len))
        words_line.append('{:{align}{width}}'.format(token, align='^', width=max_len))
        
    # выводим результат    
    print(' '.join(tags_line))
    print(' '.join(words_line))

# Пример использования функции
pos_info(recipe_steps_df.loc[241106]['name'])

   JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives
